In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from scipy.signal import convolve2d
from sklearn.metrics import classification_report
import os

# Custom callback to capture convolution results after each epoch
class CaptureConvolutionCallback(tf.keras.callbacks.Callback):
    def __init__(self, x_image, filters, save_path, epochs):
        self.x_image = x_image
        self.filters = filters
        self.save_path = save_path
        self.epochs = epochs
        self.epoch_images = []

    def on_epoch_end(self, epoch, logs=None):
        conv_output_1 = self.apply_convolution(self.x_image, self.filters[0][:, :, 0, 0])
        conv_output_2 = self.apply_convolution(self.x_image, self.filters[0][:, :, 0, 1])
        self.epoch_images.append((conv_output_1, conv_output_2))
        
        # Update filters
        for layer in self.model.layers:
            if 'conv' in layer.name:
                self.filters[0], _ = layer.get_weights()

        if epoch == self.epochs - 1:
            self.save_convolved_images()

    def apply_convolution(self, image, kernel):
        return convolve2d(image.squeeze(), kernel, mode='same', boundary='fill', fillvalue=0)

    def save_convolved_images(self):
        fig, axes = plt.subplots(self.epochs + 1, 3, figsize=(15, 5 * (self.epochs + 1)))
        
        # Plot original image
        axes[0, 0].imshow(self.x_image.reshape(28, 28), cmap='gray')
        axes[0, 0].set_title('Original Image')
        axes[0, 0].axis('off')

        # Plot initial convolutions
        axes[0, 1].imshow(self.epoch_images[0][0], cmap='gray')
        axes[0, 1].set_title('Initial Convolution - Filter 1')
        axes[0, 1].axis('off')

        axes[0, 2].imshow(self.epoch_images[0][1], cmap='gray')
        axes[0, 2].set_title('Initial Convolution - Filter 2')
        axes[0, 2].axis('off')

        # Plot convolutions after each epoch
        for i in range(self.epochs):
            axes[i + 1, 0].imshow(self.x_image.reshape(28, 28), cmap='gray')
            axes[i + 1, 0].set_title(f'Epoch {i + 1} - Original Image')
            axes[i + 1, 0].axis('off')

            axes[i + 1, 1].imshow(self.epoch_images[i][0], cmap='gray')
            axes[i + 1, 1].set_title(f'Epoch {i + 1} - Convolution - Filter 1')
            axes[i + 1, 1].axis('off')

            axes[i + 1, 2].imshow(self.epoch_images[i][1], cmap='gray')
            axes[i + 1, 2].set_title(f'Epoch {i + 1} - Convolution - Filter 2')
            axes[i + 1, 2].axis('off')

        plt.tight_layout()
        plt.savefig(self.save_path)
        plt.close()


# Load MNIST data
mnist = tf.keras.datasets.mnist
(x_full, y_full), (x_test, y_test) = mnist.load_data()

# Selecting 5 samples each from digit '0' and '1'
x_short = np.concatenate([x_full[y_full == 0][:5], x_full[y_full == 1][:5]])
y_short = np.concatenate([y_full[y_full == 0][:5], y_full[y_full == 1][:5]])
indices = np.arange(x_short.shape[0])
np.random.shuffle(indices)
x_short = x_short[indices]
y_short = y_short[indices]
x_test_short = np.concatenate([x_test[y_test == 0][:5], x_test[y_test == 1][:5]])
y_test_short = np.concatenate([y_test[y_test == 0][:5], y_test[y_test == 1][:5]])
indices_test = np.arange(x_test_short.shape[0])
np.random.shuffle(indices_test)
x_test_short = x_test_short[indices_test]
y_test_short = y_test_short[indices_test]

# Reshape x to add a channel dimension
x_short = x_short.reshape(-1, 28, 28, 1)
x_test_short = x_test_short.reshape(-1, 28, 28, 1)

# Build the CNN model
model = tf.keras.models.Sequential([
    layers.Conv2D(2, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(169, activation='relu'),
    layers.Dense(13, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Initial filters (before training)
initial_filters = []
for layer in model.layers:
    if 'conv' in layer.name:
        a, b = layer.get_weights()
        initial_filters.append(a)

# Save convolved images after each epoch
capture_conv_callback = CaptureConvolutionCallback(x_short[0], initial_filters, 'convolved_images.png', epochs=5)

# Train the model
model.fit(x_short, y_short, epochs=5, callbacks=[capture_conv_callback])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test_short, y_test_short)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Predict and generate classification report
predictions = model.predict(x_test_short)
y_pred = (predictions > 0.5).astype(int)
print(classification_report(y_test_short, y_pred))


Epoch 1/5


c:\Users\nandi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 739ms/step - accuracy: 0.4000 - loss: 14.0588
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4000 - loss: 9.0107
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5000 - loss: 5.6907
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6000 - loss: 2.7290
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 922ms/step - accuracy: 0.8000 - loss: 0.9999
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.6000 - loss: 3.9392
Test Loss: 3.9391753673553467
Test Accuracy: 0.6000000238418579
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
              precision    recall  f1-score   support

           0       0.60      0.60      0.60         5
           1       0.60      0.60      0.60         5

    accuracy                           0.60        10
   macro avg       0.60      0.60      0.60        10
weighted avg       0.60      0.60      0.60        10

